In [3]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, SVR
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, \
RocCurveDisplay, roc_auc_score, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor, BaggingClassifier, BaggingRegressor
from sklearn.linear_model import ridge_regression, ElasticNet
from sklearn.linear_model import Ridge


In [4]:
df = pd.read_csv('../datasets/HR_comma_sep.csv')
df

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [5]:
ohe = OneHotEncoder(drop = 'first', sparse_output = False)
col_trnf = ColumnTransformer([('OHE', ohe,make_column_selector(dtype_include=object) )],
                             remainder='passthrough',
                             verbose_feature_names_out=False)

col_trnf = col_trnf.set_output(transform = 'pandas')

In [6]:
x,y = df.drop('left', axis=1), df['left']

x = col_trnf.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=25, test_size=0.3, stratify=y)


Logistic Regression

In [7]:
import warnings
warnings.filterwarnings('ignore')

lr = LogisticRegression()

bagg = BaggingClassifier(random_state=25, estimator = lr, n_estimators=15)
bagg.fit(x_train, y_train)
y_pred = bagg.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.8068459657701712


bagging regressors


In [ ]:
svm = SVC(kernel = 'rbf')
scaler = StandardScaler()
knn = KNeighborsRegressor(n_neighbors=5)
pipe_svm = Pipeline([('Scaling', scaler),('SVM', svm)])
pipe_knn = Pipeline([('Scaling', scaler),('KNN', knn)])
dtc1 = DecisionTreeRegressor(random_state=25, max_depth=4)
dtc2 = DecisionTreeRegressor(random_state=25)

estimators = [dtc1, dtc2, lr, pipe_knn, pipe_svm]
scores = []
for e in tqdm(estimators):
    bagg = BaggingRegressor(random_state=25, estimator = e, n_estimators=15)
    bagg.fit(x_train, y_train)
    y_pred = bagg.predict(x_test)
    scores.append([e,mean_absolute_error(y_test, y_pred)])
    # print(accuracy_score(y_test, y_pred))
df_scores = pd.DataFrame(scores , columns = ['Estimator', 'mean_absolute_error'])
df_scores.sort_values('mean_absolute_error', ascending=True)

100%|██████████| 5/5 [00:42<00:00,  8.46s/it]


,Estimator,accuracyScore
2,LogisticRegression(),0.195555
3,"(StandardScaler(), KNeighborsRegressor())",0.068611
0,"DecisionTreeRegressor(max_depth=4, random_stat...",0.057443
4,"(StandardScaler(), SVC())",0.051300
1,DecisionTreeRegressor(random_state=25),0.027162


trying with different n_estimatores

In [ ]:
n_estimators = [10,15,25,50,100]
svm = SVR(kernel = 'rbf')
estimators = [dtc1, dtc2, lr, pipe_knn, pipe_svm]
scores = []
for e in tqdm(estimators):
    for n in n_estimators:
        bagg = BaggingRegressor(random_state=25, estimator = e, n_estimators=n)
        bagg.fit(x_train, y_train)
        y_pred = bagg.predict(x_test)
        scores.append([n,e,mean_absolute_error(y_test, y_pred)])
    # print(accuracy_score(y_test, y_pred))
df_scores = pd.DataFrame(scores , columns = ['n_estimators','Estimator', 'mean_absolute_error'])
df_scores.sort_values('mean_absolute_error', ascending=True)

100%|██████████| 5/5 [08:44<00:00, 104.91s/it]


,n_estimators,Estimator,accuracyScore
9,100,DecisionTreeRegressor(random_state=25),0.026281
8,50,DecisionTreeRegressor(random_state=25),0.026317
7,25,DecisionTreeRegressor(random_state=25),0.026628
6,15,DecisionTreeRegressor(random_state=25),0.027162
5,10,DecisionTreeRegressor(random_state=25),0.027473
23,50,"(StandardScaler(), SVC())",0.050629
24,100,"(StandardScaler(), SVC())",0.050729
22,25,"(StandardScaler(), SVC())",0.050838
21,15,"(StandardScaler(), SVC())",0.051300
20,10,"(StandardScaler(), SVC())",0.051789
